# Machine Learning to Identify Forams species and and whether the shell is cracked or not

## Finnegan Lab
## Data preprocessing

### Setup

In [1]:
# Imports. Feel free to add or remove as necessary 

# !pip install -Uq tensorflow-datasets==4.5.2
# !pip install tf-nightly -q
import pandas as pd
import numpy as np
np.random.seed(2022)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import os, re, math
import keras
import shutil
from pathlib import Path
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds

%load_ext autoreload
%autoreload 2

In [2]:
#@title mount cloud
from google.colab import auth
auth.authenticate_user()
project_id = 'lucid-loader-347021'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://paleo-ml/


In [3]:
from google.colab import drive
drive.mount('/content/drive')
dir_drive_code ='drive/MyDrive/MV1012_SBB_images/ML_projects/Code'
assert os.path.isdir(dir_drive_code), 'unable to find code. Please check your drive path above.'

Mounted at /content/drive


In [4]:
!git clone https://github.com/BryanZWu/finnegan-paleo-ml.git
dir_code = 'finnegan-paleo-ml/Code'
!cd {dir_code}
os.chdir(dir_code)

Cloning into 'finnegan-paleo-ml'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 114 (delta 42), reused 113 (delta 41), pack-reused 0
Receiving objects: 100% (114/114), 36.35 KiB | 715.00 KiB/s, done.
Resolving deltas: 100% (42/42), done.


In [5]:
# Data stored in drive specified here
dir_base = '/content/drive/MyDrive/MV1012_SBB_images/'
dir_raw_training_images = f'{dir_base}/Box_Core_images'
dir_ml_home = f'{dir_base}/ML_projects'
dir_google_cloud = 'gs://paleo-ml'
dir_processed_data = dir_out_data = f'{dir_ml_home}/Processed_data/forams'
dir_training_logs = f'{dir_google_cloud}/Training_logs/'
dir_trained_models = f'{dir_ml_home}/Trained_models/'
dir_train_data = f'{dir_processed_data}/train'
dir_val_data = f'{dir_processed_data}/val'
dir_test_data = f'{dir_processed_data}/test'
dir_debug_processed_data = '/content/training_data_debug'
dir_labels_csv = f'{dir_base}/Final_Diversity_CSVs/ML_master2.csv'

In [6]:
#@title import local project files
from common.constants import *
from common.imports import *
from data_processing import data_processor

In [7]:
def git():
    """ Refetch all files """
    !git fetch
    !git checkout origin -- .

## Processing data and storing

Take data from its original form and crop/resize images as necessary. Store images with its corresponding label.

### Run processing

In [8]:
labels_df = data_processor.create_labels_df(dir_labels_csv)
labels_df.head()


Broken            species     age   test    val
sample_name  object_num                                                   
MV1012-BC-12 1.0         unbroken          NOT FORAM  1994.0  False  False
             2.0         unbroken   suggrunda eckisi  1994.0  False  False
             3.0           broken    bulimina exilis  1994.0  False  False
             4.0         unbroken  nonionella stella  1994.0  False  False
             5.0         unbroken          NOT FORAM  1994.0  False   True

In [ ]:
#Run data processing
data_processor.run_processing(dir_debug_processed_data, dir_raw_training_images, labels_df)

## Create tensorflow dataset and push to google cloud

In [30]:
git()

remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 6 (delta 2), reused 6 (delta 2), pack-reused 0
Unpacking objects: 100% (6/6), done.
From https://github.com/BryanZWu/finnegan-paleo-ml
 + 12a3f39...aed539e main       -> origin/main  (forced update)


In [31]:
data_processor.create_cloud_dataset(dir_debug_processed_data, dir_google_cloud, 256)

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

FileNotFoundError: ignored